In [13]:
from textblob import TextBlob
#import spacy
import pandas as pd
import numpy as np
import re
import spacy
import yake
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj= SentimentIntensityAnalyzer()
import nltk
#nltk.download('omw-1.4')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


In [40]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now", "go", "get", "think", "know", "say", "says", "want"]


#from nltk.corpus import stopwords
#stop = stopwords.words('english')

In [41]:
names = ['id', 'date_publish', 'outlet', 'headline', 'lead', 'body', 'authors', 'domain', 'url', 'political_leaning']
data = pd.read_csv('clean_climate_data.csv',encoding='latin1', usecols=names)
df = pd.DataFrame(data)
# Make sure the following columns are all strings
df['body'] = df['body'].astype(str)
df['lead'] = df['lead'].astype(str)
df['headline'] = df['headline'].astype(str)
df.head()

,id,date_publish,outlet,headline,lead,body,authors,domain,url,political_leaning
0,116016630,2017-01-01 00:00:00,ABC News,New UN Chief Urges New Year's Resolution: 'Put...,New UN Chief Urges New Year's Resolution: 'Put...,Antonio Guterres took the reins of the United ...,Abc News,abcnews.go.com,http://abcnews.go.com/US/wireStory/chief-urges...,CENTER
1,39183262,2017-01-01 00:04:40,The Guardian,The Observer view on the prospects for 2017,"Europe, Trump, the Middle EastÃ¢ÂÂ¦ a year of...",It is the conceit of every generation to belie...,Observer Editorial,www.theguardian.com,https://www.theguardian.com/commentisfree/2017...,LEFT
2,39113103,2017-01-01 00:05:41,The Guardian,The moderate insurgency: A better world can em...,A backlash to a year of extremes and polarisat...,"After the events of 2016, foreseeing the year ...",Matthew Taylor,www.theguardian.com,https://www.theguardian.com/commentisfree/2017...,LEFT
3,4633089,2017-01-01 06:17:27,Fox News,"Spread by trade and climate, bugs butcher Amer...",In a towering forest of centuries-old eastern ...,In a towering forest of centuries-old eastern ...,NaN,www.foxnews.com,http://www.foxnews.com/us/2017/01/01/spread-by...,RIGHT
4,39150219,2017-01-01 07:00:49,The Guardian,Will AntÃÂ³nio Guterres be the UN's best ever...,"Cultured and consensual, the Portuguese politi...",When AntÃÂ³nio Guterres resigned halfway thro...,Angelique Chrisafis;Julian Borger,www.theguardian.com,https://www.theguardian.com/world/2017/jan/01/...,LEFT


In [46]:
# 1) Remove hyperlinks 2) Remove emails
df['processed_body'] = df['body'].replace('\S*@\S*\s?', ' ', regex=True)
df['processed_body'] = df['processed_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
# 3) Make everything on a single line 4) Remove non-alphanumeric characters 
df['processed_body'] = df['processed_body'].replace(r'\n',' ', regex=True) 
df['processed_body'] = df['processed_body'].replace('[^a-zA-Z0-9 ]', ' ', regex=True)
# 5) Convert to lowercase 6) Remove stopwords
df['processed_body'] = df['processed_body'].apply(lambda s: s.lower() if type(s) == str else s)
df['processed_body'] = df['processed_body'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))


In [48]:
df['processed_body'].str.contains(" say ")
#print(df['body'][3])
print(df['processed_body'][3])

towering forest centuries old eastern hemlocks easy miss one tree nemeses larger speck pepper hemlock woolly adelgid spends life underside needles sucking sap eventually killing tree bug one expanding army insects draining life forests new england west coast aided global trade warming climate drought weakened trees invaders become one greatest threats biodiversity united states scientists already driving tree species toward extinction causing billions dollars year damage situation expected worsen one things actually eliminate forest tree species pretty short order within years said harvard university ecologist david orwig walked past dead hemlocks scattered across university 5 8 square mile research forest petersham scourge projected put 63 percent country forest risk 2027 carries cost several billion dollars annually dead tree removal declining property values timber industry losses according peer reviewed study year ecological applications examination dozen experts found hundreds pes

In [49]:
# Warning: This may take a few minutes to run, but it does terminate (with vader added it takes almost a half hour smh)
tb_polarity_df = []
tb_subjectivity_df = []
vs_polarity_df = []

for index, row in df.iterrows():
    text = row["headline"]
    blob = TextBlob(text)
    #print(blob.polarity)
    tb_polarity_df.append(blob.polarity)
    tb_subjectivity_df.append(blob.subjectivity)
    vs_polarity_df.append(sid_obj.polarity_scores(text))
    #print(row["body"])
    #print("\n")

    
tb_polarity_df = pd.DataFrame(tb_polarity_df)
tb_subjectivity_df = pd.DataFrame(tb_subjectivity_df)
vs_polarity_df = pd.DataFrame(vs_polarity_df)
print(tb_polarity_df.head(),'\n', tb_subjectivity_df.head(),'\n', vs_polarity_df['compound'].head())

#for sentence in blob.sentences:
#    print(sentence.sentiment.polarity)

          0
0  0.174242
1  0.000000
2  0.250000
3  0.000000
4  0.525000 
           0
0  0.414141
1  0.000000
2  0.600000
3  0.000000
4  0.400000 
 0    0.5423
1    0.2960
2   -0.0258
3    0.0000
4    0.6369
Name: compound, dtype: float64


In [50]:
# Add to dataframe and write to a new CSV file
df['textblob_polarity'] = tb_polarity_df
df['textblob_subjectivity'] = tb_subjectivity_df
df['vader_polarity_negative'] = vs_polarity_df.neg
df['vader_polarity_neutral'] = vs_polarity_df.neu
df['vader_polarity_positive'] = vs_polarity_df.pos
df['vader_polarity_compound'] = vs_polarity_df.compound





In [51]:
df

,id,date_publish,outlet,headline,lead,body,authors,domain,url,political_leaning,processed_body,textblob_polarity,textblob_subjectivity,vader_polarity_negative,vader_polarity_neutral,vader_polarity_positive,vader_polarity_compound
0,116016630,2017-01-01 00:00:00,ABC News,New UN Chief Urges New Year's Resolution: 'Put...,New UN Chief Urges New Year's Resolution: 'Put...,Antonio Guterres took the reins of the United ...,Abc News,abcnews.go.com,http://abcnews.go.com/US/wireStory/chief-urges...,CENTER,antonio guterres took reins united nations new...,0.174242,0.414141,0.000,0.720,0.280,0.5423
1,39183262,2017-01-01 00:04:40,The Guardian,The Observer view on the prospects for 2017,"Europe, Trump, the Middle EastÃ¢ÂÂ¦ a year of...",It is the conceit of every generation to belie...,Observer Editorial,www.theguardian.com,https://www.theguardian.com/commentisfree/2017...,LEFT,conceit every generation believe experience un...,0.000000,0.000000,0.000,0.761,0.239,0.2960
2,39113103,2017-01-01 00:05:41,The Guardian,The moderate insurgency: A better world can em...,A backlash to a year of extremes and polarisat...,"After the events of 2016, foreseeing the year ...",Matthew Taylor,www.theguardian.com,https://www.theguardian.com/commentisfree/2017...,LEFT,events 2016 foreseeing year ahead may foolhard...,0.250000,0.600000,0.178,0.651,0.172,-0.0258
3,4633089,2017-01-01 06:17:27,Fox News,"Spread by trade and climate, bugs butcher Amer...",In a towering forest of centuries-old eastern ...,In a towering forest of centuries-old eastern ...,NaN,www.foxnews.com,http://www.foxnews.com/us/2017/01/01/spread-by...,RIGHT,towering forest centuries old eastern hemlocks...,0.000000,0.000000,0.000,1.000,0.000,0.0000
4,39150219,2017-01-01 07:00:49,The Guardian,Will AntÃÂ³nio Guterres be the UN's best ever...,"Cultured and consensual, the Portuguese politi...",When AntÃÂ³nio Guterres resigned halfway thro...,Angelique Chrisafis;Julian Borger,www.theguardian.com,https://www.theguardian.com/world/2017/jan/01/...,LEFT,ant nio guterres resigned halfway second term ...,0.525000,0.400000,0.000,0.682,0.318,0.6369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26657,3867493,2019-08-31 16:05:32,HuffPost,Brazil's Bolsonaro Vows To Stop Using Bic Pens...,The Brazilian president says he's dumping the ...,"As raging fires rip through the Amazon, Brazil...",Trends Reporter,www.huffpost.com,https://www.huffpost.com/entry/brazil-jair-bol...,LEFT,raging fires rip amazon brazilian president ja...,0.000000,0.000000,0.155,0.845,0.000,-0.2960
26658,39019044,2019-08-31 17:00:53,The Guardian,Fears for BritainÃ¢ÂÂs standing in world of ...,Business leaders urge government to take actio...,Business leaders urge government to take actio...,Robin Mckie,www.theguardian.com,https://www.theguardian.com/education/2019/aug...,LEFT,business leaders urge government take action u...,0.000000,0.000000,0.189,0.811,0.000,-0.4215
26659,4161060,2019-08-31 18:22:49,USA Today,Trump administration moves to expand electric ...,Supporters of the change say it rightly gives ...,"CLOSE Trade war with China causes price hikes,...",Ledyard King;Published P.M. Et Aug.,www.usatoday.com,https://www.usatoday.com/story/news/politics/2...,CENTER,close trade war china causes price hikes uncer...,0.000000,0.000000,0.000,0.796,0.204,0.3182
26660,4331363,2019-08-31 19:05:37,Breitbart,China Claims Trade War Hinders Efforts to Cont...,Communist China claims its trade war with the ...,The Chinese Ministry of Ecology and Environmen...,John Hayward,www.breitbart.com,https://www.breitbart.com/national-security/20...,RIGHT,chinese ministry ecology environment claimed f...,0.000000,0.000000,0.302,0.698,0.000,-0.5994


In [52]:
tb_neg_df = df[df['textblob_polarity'] < 0]
tb_pos_df = df[df['textblob_polarity'] > 0]
tb_obj_df = df[df['textblob_subjectivity'] < 0.5]
tb_subj_df = df[df['textblob_subjectivity'] > 0.5]
vs_neg_df = df[df['vader_polarity_compound'] < 0]
vs_pos_df = df[df['vader_polarity_compound'] > 0]

print("Number of negative articles from textblob: ", len(tb_neg_df))
print("Number of positive articles from textblob: ", len(tb_pos_df))
print("Number of objective articles from textblob: ", len(tb_obj_df))
print("Number of subjective articles from textblob: ", len(tb_subj_df))
print("Number of negative articles from VADER: ", len(vs_neg_df))
print("Number of positive articles from VADER: ", len(vs_pos_df))


# Get the count of each article, sorted alphabetically
total_counts = df.outlet.value_counts().sort_index(ascending=True)
neg_counts = tb_neg_df.outlet.value_counts().sort_index(ascending=True)
pos_counts = tb_pos_df.outlet.value_counts().sort_index(ascending=True)
obj_counts = tb_obj_df.outlet.value_counts().sort_index(ascending=True)
subj_counts = tb_subj_df.outlet.value_counts().sort_index(ascending=True)
vs_neg_counts = vs_neg_df.outlet.value_counts().sort_index(ascending=True)
vs_pos_counts = vs_pos_df.outlet.value_counts().sort_index(ascending=True)
print('\n')

print("Rates of negative articles:\n")
print((neg_counts/total_counts).sort_values(ascending=False), '\n\n')

print("Rates of positive articles:\n")
print((pos_counts/total_counts).sort_values(ascending=False), '\n\n')

print("Rates of objective articles:\n")
print((obj_counts/total_counts).sort_values(ascending=False), '\n\n')

print("Rates of subjective articles:\n")
print((subj_counts/total_counts).sort_values(ascending=False), '\n\n')
#print(neg_df.outlet.value_counts().sort_index(ascending=True)/df.outlet.value_counts().sort_index(ascending=True), '\n')

print("Rates of negative articles using VADER:\n")
print((vs_neg_counts/total_counts).sort_values(ascending=False), '\n\n')

print("Rates of positive articles using VADER:\n")
print((vs_pos_counts/total_counts).sort_values(ascending=False), '\n\n')


Number of negative articles from textblob:  4002
Number of positive articles from textblob:  6698
Number of objective articles from textblob:  21278
Number of subjective articles from textblob:  4015
Number of negative articles from VADER:  9414
Number of positive articles from VADER:  6945


Rates of negative articles:

Fox News                   0.194304
Breitbart                  0.193108
NBC News                   0.166876
The Guardian               0.166290
HuffPost                   0.165217
Los Angeles Times          0.158438
Slate                      0.154723
Chicago Tribune            0.142672
Reuters                    0.140818
The New York Times         0.133142
USA Today                  0.129559
CNN                        0.126437
CBS News                   0.124642
Politico                   0.124398
NPR                        0.117593
The Wall Street Journal    0.111111
ABC News                   0.111111
BBC                        0.101707
Name: outlet, dtype: float64 

In [53]:
gwsd_data = pd.read_csv("predictions_test_headline.tsv", delimiter="\t")
gwsd_df = pd.DataFrame(gwsd_data)
gwsd_df = pd.concat([df, gwsd_df], axis=1)
neutral_df = gwsd_df[gwsd_df['predicted'] == 1]
for_df = gwsd_df[gwsd_df['predicted'] == 2]
against_df = gwsd_df[gwsd_df['predicted'] == 0]

neutral_counts = neutral_df.outlet.value_counts().sort_index(ascending=True)
for_counts = for_df.outlet.value_counts().sort_index(ascending=True)
against_counts = against_df.outlet.value_counts().sort_index(ascending=True)


print("Rates of articles neutral to climate change:\n")
print((neutral_counts/total_counts).sort_values(ascending=False), '\n\n')

print("Rates of articles supporting climate change:\n")
print((for_counts/total_counts).sort_values(ascending=False), '\n\n')

print("Rates of articles against climate change:\n")
print((against_counts/total_counts).sort_values(ascending=False), '\n\n')
gwsd_df

Rates of articles neutral to climate change:

Politico                   0.926164
ABC News                   0.905724
Reuters                    0.901047
NBC News                   0.879548
CNN                        0.878868
Fox News                   0.874684
NPR                        0.873291
CBS News                   0.859599
Los Angeles Times          0.855564
Chicago Tribune            0.853437
The New York Times         0.848246
BBC                        0.846373
The Wall Street Journal    0.843434
USA Today                  0.831094
Breitbart                  0.791816
HuffPost                   0.782971
The Guardian               0.774665
Slate                      0.765472
Name: outlet, dtype: float64 


Rates of articles supporting climate change:

The Guardian               0.129236
CBS News                   0.113181
HuffPost                   0.109058
BBC                        0.108819
USA Today                  0.107486
NPR                        0.096627
Los Angeles 

,id,date_publish,outlet,headline,lead,body,authors,domain,url,political_leaning,...,textblob_subjectivity,vader_polarity_negative,vader_polarity_neutral,vader_polarity_positive,vader_polarity_compound,true,predicted,0,1,2
0,116016630,2017-01-01 00:00:00,ABC News,New UN Chief Urges New Year's Resolution: 'Put...,New UN Chief Urges New Year's Resolution: 'Put...,Antonio Guterres took the reins of the United ...,Abc News,abcnews.go.com,http://abcnews.go.com/US/wireStory/chief-urges...,CENTER,...,0.414141,0.000,0.720,0.280,0.5423,2,2,-1.911445,0.786068,1.995868
1,39183262,2017-01-01 00:04:40,The Guardian,The Observer view on the prospects for 2017,"Europe, Trump, the Middle EastÃ¢ÂÂ¦ a year of...",It is the conceit of every generation to belie...,Observer Editorial,www.theguardian.com,https://www.theguardian.com/commentisfree/2017...,LEFT,...,0.000000,0.000,0.761,0.239,0.2960,2,1,-1.099629,1.978928,-0.501376
2,39113103,2017-01-01 00:05:41,The Guardian,The moderate insurgency: A better world can em...,A backlash to a year of extremes and polarisat...,"After the events of 2016, foreseeing the year ...",Matthew Taylor,www.theguardian.com,https://www.theguardian.com/commentisfree/2017...,LEFT,...,0.600000,0.178,0.651,0.172,-0.0258,2,1,-0.601929,0.575598,-0.202554
3,4633089,2017-01-01 06:17:27,Fox News,"Spread by trade and climate, bugs butcher Amer...",In a towering forest of centuries-old eastern ...,In a towering forest of centuries-old eastern ...,NaN,www.foxnews.com,http://www.foxnews.com/us/2017/01/01/spread-by...,RIGHT,...,0.000000,0.000,1.000,0.000,0.0000,2,1,-0.573077,1.225463,-0.821509
4,39150219,2017-01-01 07:00:49,The Guardian,Will AntÃÂ³nio Guterres be the UN's best ever...,"Cultured and consensual, the Portuguese politi...",When AntÃÂ³nio Guterres resigned halfway thro...,Angelique Chrisafis;Julian Borger,www.theguardian.com,https://www.theguardian.com/world/2017/jan/01/...,LEFT,...,0.400000,0.000,0.682,0.318,0.6369,2,1,-1.311096,1.721820,0.225075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26657,3867493,2019-08-31 16:05:32,HuffPost,Brazil's Bolsonaro Vows To Stop Using Bic Pens...,The Brazilian president says he's dumping the ...,"As raging fires rip through the Amazon, Brazil...",Trends Reporter,www.huffpost.com,https://www.huffpost.com/entry/brazil-jair-bol...,LEFT,...,0.000000,0.155,0.845,0.000,-0.2960,2,1,-0.273092,1.411716,-1.544832
26658,39019044,2019-08-31 17:00:53,The Guardian,Fears for BritainÃ¢ÂÂs standing in world of ...,Business leaders urge government to take actio...,Business leaders urge government to take actio...,Robin Mckie,www.theguardian.com,https://www.theguardian.com/education/2019/aug...,LEFT,...,0.000000,0.189,0.811,0.000,-0.4215,2,1,0.292045,0.873179,-1.464980
26659,4161060,2019-08-31 18:22:49,USA Today,Trump administration moves to expand electric ...,Supporters of the change say it rightly gives ...,"CLOSE Trade war with China causes price hikes,...",Ledyard King;Published P.M. Et Aug.,www.usatoday.com,https://www.usatoday.com/story/news/politics/2...,CENTER,...,0.000000,0.000,0.796,0.204,0.3182,2,1,-0.984110,2.063841,-0.998478
26660,4331363,2019-08-31 19:05:37,Breitbart,China Claims Trade War Hinders Efforts to Cont...,Communist China claims its trade war with the ...,The Chinese Ministry of Ecology and Environmen...,John Hayward,www.breitbart.com,https://www.breitbart.com/national-security/20...,RIGHT,...,0.000000,0.302,0.698,0.000,-0.5994,2,1,-0.443737,1.519115,-1.267052


In [54]:
# p_threshold/s_treshold is the neutrality threshold for which a value to still be neutral 
# p_treshold is for polarity and should be between 0 and 1, but close to 0 
# s_threshold is for subjectivity and can be between 0 and 1, but should be close to 0.5 polarity, s=subjectivity)
# Example: p_threshold=0.05 counts -0.05<x<0.05 as neutral
def make_pred(p_threshold, s_threshold, df):
    df['gwsd_pred'] = gwsd_df['predicted']
    df['vader_polarity_pred'] = [0 if x<-p_threshold else 1 if -p_threshold<=x<p_threshold else 2 for x in df['vader_polarity_compound']]
    df['textblob_polarity_pred'] = [0 if x<-p_threshold else 1 if -p_threshold<=x<p_threshold else 2 for x in df['textblob_polarity']]
    df['textblob_subjectivity_pred'] = [0 if x<=s_threshold else 1 for x in df['textblob_subjectivity']]
    return df
    
    
df = make_pred(0.05, 0.5, df)    
df.to_csv('climate_data_sentiment_headlines.csv', index=False)
df

,id,date_publish,outlet,headline,lead,body,authors,domain,url,political_leaning,...,textblob_polarity,textblob_subjectivity,vader_polarity_negative,vader_polarity_neutral,vader_polarity_positive,vader_polarity_compound,gwsd_pred,vader_polarity_pred,textblob_polarity_pred,textblob_subjectivity_pred
0,116016630,2017-01-01 00:00:00,ABC News,New UN Chief Urges New Year's Resolution: 'Put...,New UN Chief Urges New Year's Resolution: 'Put...,Antonio Guterres took the reins of the United ...,Abc News,abcnews.go.com,http://abcnews.go.com/US/wireStory/chief-urges...,CENTER,...,0.174242,0.414141,0.000,0.720,0.280,0.5423,2,2,2,0
1,39183262,2017-01-01 00:04:40,The Guardian,The Observer view on the prospects for 2017,"Europe, Trump, the Middle EastÃ¢ÂÂ¦ a year of...",It is the conceit of every generation to belie...,Observer Editorial,www.theguardian.com,https://www.theguardian.com/commentisfree/2017...,LEFT,...,0.000000,0.000000,0.000,0.761,0.239,0.2960,1,2,1,0
2,39113103,2017-01-01 00:05:41,The Guardian,The moderate insurgency: A better world can em...,A backlash to a year of extremes and polarisat...,"After the events of 2016, foreseeing the year ...",Matthew Taylor,www.theguardian.com,https://www.theguardian.com/commentisfree/2017...,LEFT,...,0.250000,0.600000,0.178,0.651,0.172,-0.0258,1,1,2,1
3,4633089,2017-01-01 06:17:27,Fox News,"Spread by trade and climate, bugs butcher Amer...",In a towering forest of centuries-old eastern ...,In a towering forest of centuries-old eastern ...,NaN,www.foxnews.com,http://www.foxnews.com/us/2017/01/01/spread-by...,RIGHT,...,0.000000,0.000000,0.000,1.000,0.000,0.0000,1,1,1,0
4,39150219,2017-01-01 07:00:49,The Guardian,Will AntÃÂ³nio Guterres be the UN's best ever...,"Cultured and consensual, the Portuguese politi...",When AntÃÂ³nio Guterres resigned halfway thro...,Angelique Chrisafis;Julian Borger,www.theguardian.com,https://www.theguardian.com/world/2017/jan/01/...,LEFT,...,0.525000,0.400000,0.000,0.682,0.318,0.6369,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26657,3867493,2019-08-31 16:05:32,HuffPost,Brazil's Bolsonaro Vows To Stop Using Bic Pens...,The Brazilian president says he's dumping the ...,"As raging fires rip through the Amazon, Brazil...",Trends Reporter,www.huffpost.com,https://www.huffpost.com/entry/brazil-jair-bol...,LEFT,...,0.000000,0.000000,0.155,0.845,0.000,-0.2960,1,0,1,0
26658,39019044,2019-08-31 17:00:53,The Guardian,Fears for BritainÃ¢ÂÂs standing in world of ...,Business leaders urge government to take actio...,Business leaders urge government to take actio...,Robin Mckie,www.theguardian.com,https://www.theguardian.com/education/2019/aug...,LEFT,...,0.000000,0.000000,0.189,0.811,0.000,-0.4215,1,0,1,0
26659,4161060,2019-08-31 18:22:49,USA Today,Trump administration moves to expand electric ...,Supporters of the change say it rightly gives ...,"CLOSE Trade war with China causes price hikes,...",Ledyard King;Published P.M. Et Aug.,www.usatoday.com,https://www.usatoday.com/story/news/politics/2...,CENTER,...,0.000000,0.000000,0.000,0.796,0.204,0.3182,1,2,1,0
26660,4331363,2019-08-31 19:05:37,Breitbart,China Claims Trade War Hinders Efforts to Cont...,Communist China claims its trade war with the ...,The Chinese Ministry of Ecology and Environmen...,John Hayward,www.breitbart.com,https://www.breitbart.com/national-security/20...,RIGHT,...,0.000000,0.000000,0.302,0.698,0.000,-0.5994,1,0,1,0


In [37]:
kw_extractor = yake.KeywordExtractor()
text = """spaCy is an open-source software library for advanced natural language processing, written in the programming languages Python and Cython. The library is published under the MIT license and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
numOfKeywords = 15
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
    print(kw)

('programming languages Python', 0.001295347548560416)
('natural language processing', 0.002012136772192602)
('advanced natural language', 0.0026621455770583914)
('Python and Cython', 0.0035840985079775055)
('open-source software library', 0.008298152696966859)
('languages Python', 0.009390717577572831)
('language processing', 0.01453240965208459)
('software company Explosion', 0.015993140254256993)
('advanced natural', 0.01840251352140607)
('natural language', 0.019161829017826378)
('programming languages', 0.019161829017826378)
('open-source software', 0.032652195076937375)
('Ines Montani', 0.03375876229391358)
('Matthew Honnibal', 0.04096703831447956)
('Honnibal and Ines', 0.04096703831447956)


In [55]:

headline_keyword_df = []
lead_keyword_df = []

for index, row in df.iterrows():
    headline_text = row["headline"]
    lead_text = row["lead"]
    headline_keywords = custom_kw_extractor.extract_keywords(headline_text)
    #print(type(lead_text))
    lead_keywords = custom_kw_extractor.extract_keywords(lead_text)
    #print(keywords)
    headline_keyword_df.append(headline_keywords)
    lead_keyword_df.append(lead_keywords)
    

headline_keyword_df = pd.DataFrame(headline_keyword_df)
lead_keyword_df = pd.DataFrame(lead_keyword_df)
#lead_keyword_df

#print(df[15])

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

...

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [56]:
#headline_keyword_df.to_csv('headline_keywords.csv')
#lead_keyword_df.to_csv('lead_keywords.csv')

In [4]:
body = pd.DataFrame(df['body'])

#for index, row in body.iterrows():
#    row = row.replace("\n\r", " ")
    #print(row)
    
body = body.replace(r'\n',' ', regex=True) 
body.insert(1, 'label', 'agree')
body.insert(2, 'value', 1.0)

In [5]:
body.to_csv('test.tsv', sep="\t", header=None, index=False)

In [6]:
body

,body,label,value
0,Antonio Guterres took the reins of the United ...,agree,1.0
1,It is the conceit of every generation to belie...,agree,1.0
2,"After the events of 2016, foreseeing the year ...",agree,1.0
3,In a towering forest of centuries-old eastern ...,agree,1.0
4,When AntÃÂ³nio Guterres resigned halfway thro...,agree,1.0
...,...,...,...
26657,"As raging fires rip through the Amazon, Brazil...",agree,1.0
26658,Business leaders urge government to take actio...,agree,1.0
26659,"CLOSE Trade war with China causes price hikes,...",agree,1.0
26660,The Chinese Ministry of Ecology and Environmen...,agree,1.0
